Generate Mock Dataset

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

# Define the number of rows in the dataset
num_rows = 1000

# Generate random dates
start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2022-12-31')
date_field = pd.date_range(start_date, end_date, periods=num_rows)


# Generate random daily sales values
#daily_sales = np.random.randint(1, 10001, size=num_rows)

# Define the state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Generate random state codes
state_code = np.random.choice(state_codes, size=num_rows)

# Generate random integer fields
int_field1 = np.random.randint(1, 10001, size=num_rows)
int_field2 = np.random.randint(1, 10001, size=num_rows)
int_field3 = np.random.randint(1, 10001, size=num_rows)
int_field4 = np.random.randint(1, 10001, size=num_rows)
int_field5 = np.random.randint(1, 10001, size=num_rows)

# Create a DataFrame with the generated data
data = pd.DataFrame({
#    'Daily Sales': daily_sales,
    'StateCode': state_code,
    'var1': int_field1,
    'var2': int_field2,
    'var3': int_field3,
    'var4': int_field4,
    'var5': int_field5,
    'Date': date_field
})


data['Sales'] = 1000 + 1.2 * data['var1'] + 3.5*data['var2'] + 0.67* data['var3'] + 2.45*data['var4'] + data['var5']
data['Date'] = data['Date'].dt.strftime('%m/%d/%Y')

In [24]:
data.head()

,StateCode,var1,var2,var3,var4,var5,Date,Sales
0,MO,502,1172,8488,5795,2042,01/01/2022,27631.11
1,IL,1954,1793,6569,5330,7061,01/01/2022,34141.03
2,CA,2245,70,4315,4489,1144,01/01/2022,18972.10
3,WI,8812,1371,3434,5371,879,01/02/2022,32711.63
4,AL,2230,2761,9192,3391,4739,01/02/2022,32545.09


Model Against Sales

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Create the regression model
reg_model = LinearRegression()

# Define the independent variables
X = data[['var1', 'var2', 'var3']]

# Standardize var1, var2, and var3
scaler = StandardScaler()
X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
X1 = X[['var1s', 'var2s', 'var3s']]

# Define the dependent variable
y = data['Sales']

# Fit the model
results = reg_model.fit(X1, y)

/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_27044/1613018345.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_27044/1613018345.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['var1s', 'var2s', 'var3s']] = scaler.fit_transform(X[['var1', 'var2', 'var3']])
/var/folders/2y/295w_nh94yg378k6m37nf1x00000gn/T/ipykernel_27044/1613018345.py:12: Setti

In [42]:
# Predict the sales using the regression model
predicted_sales = reg_model.predict(X)

# Create a new DataFrame to store the predicted sales
predicted_data = data.copy()
predicted_data['Predicted Sales'] = predicted_sales

# Display the predicted sales
predicted_data[['StateCode', 'var1', 'var2', 'var3', 'Sales', 'Predicted Sales']]

,StateCode,var1,var2,var3,Sales,Predicted Sales
0,MO,502,1172,8488,27631.11,28398.502156
1,IL,1954,1793,6569,34141.03,31122.724362
2,CA,2245,70,4315,18972.10,23973.341096
3,WI,8812,1371,3434,32711.63,35643.534992
4,AL,2230,2761,9192,32545.09,36465.535564
...,...,...,...,...,...,...
995,OK,9533,7049,7536,56645.47,59134.290730
996,ID,7910,6969,5770,49852.50,55910.930608
997,NM,1770,5294,6613,37281.21,43381.145245
998,CT,6657,1039,165,30753.95,30003.147563


In [49]:
results.intercept_, results.coef_

(np.float64(45342.48222),
 array([ 3299.07367804, 10130.25439033,  1737.5830017 ]))

In [50]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# Calculate MAPE
mape = mean_absolute_percentage_error(data['Sales'], predicted_data['Predicted Sales'])

# Calculate R-squared
r_squared = r2_score(data['Sales'], predicted_data['Predicted Sales'])

mape, r_squared

(np.float64(0.16680097046026038), 0.6706462673577799)